<center><img src="https://github.com/MAGALLANESJoseManuel/OpenDataBigData_at_quality4point0/blob/main/logo.png?raw=true
" width="1000"></center>



# The Dengue DATA



1. From the official site [itself](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue)

In [ ]:
import time

link="https://www.datosabiertos.gob.pe/sites/default/files/datos_abiertos_vigilancia_dengue.csv"

import pandas as pd

# ALSO: time it takes
start_time = time.time()
dengue=pd.read_csv(link)
print(time.time() - start_time, "seconds")

2. Let's use a free cloud service (original file):


In [ ]:
start_time = time.time()

GitFolder="https://github.com/MagallanesTalks/OpenBigData_atPUCP/raw/refs/heads/main/data/"
myLinkGit=GitFolder+"datos_abiertos_vigilancia_dengue.csv"

dengue=pd.read_csv(myLinkGit, on_bad_lines="warn")

print(time.time() - start_time, "seconds")


Using a different engine:

In [ ]:
start_time = time.time()

myLinkGit=GitFolder+"datos_abiertos_vigilancia_dengue.csv"
dengueArrow=pd.read_csv(myLinkGit, on_bad_lines="warn", engine='pyarrow')

print(time.time() - start_time, "seconds")

We can recover what is missing:

In [ ]:
# badLines=["PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2009,7,A97.0,31,200701,2007010008,7,A,M;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2009,13,A97.0,31,200701,2007010008,13,A,F;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2009,9,A97.0,31,200701,2007010008,23,A,F;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2013,15,A97.0,31,200701,2007010008,15,A,F;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2013,15,A97.0,31,200701,2007010008,18,A,M;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2015,24,A97.0,31,200701,2007010008,50,A,M;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2013,7,A97.0,31,200701,2007010008,8,A,M;
# PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2017,19,A97.0,31,200701,2007010008,28,A,M"]

badLines=["PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2009,7,A97.0,31,200701,2007010008,7,A,M;PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2009,13,A97.0,31,200701,2007010008,13,A,F;PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2009,9,A97.0,31,200701,2007010008,23,A,F;PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2013,15,A97.0,31,200701,2007010008,15,A,F;PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2013,15,A97.0,31,200701,2007010008,18,A,M; PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2015,24,A97.0,31,200701,2007010008,50,A,M;PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2013,7,A97.0,31,200701,2007010008,8,A,M;PIURA,TALARA,PARIѐAS,ENACE I-II-III,DENGUE SIN SEÑALES DE ALARMA,2017,19,A97.0,31,200701,2007010008,28,A,M"]


missingLines=pd.DataFrame([cell.split(',') for cell in [ line.split(";") for line in badLines][0]],columns=dengue.columns)
missingLines

Then,

In [ ]:
dengue_ok=pd.concat([dengue,missingLines])
dengue_ok.info()

Some housecleaning:

In [ ]:
# verify the text is full of numbers
dengue_ok.loc[:,['ano','semana','edad']].apply(lambda col:col.str.isnumeric().all())

In [ ]:
# make the change
dengue_ok[['ano','semana','edad']]=dengue_ok.loc[:,['ano','semana','edad']].apply(pd.to_numeric)
dengue_ok.info()

Do we have complete location information:

In [ ]:
dengue_ok[dengue_ok.provincia.str.contains(r'[^A-Z\sÑ]',regex=True)]

Then,

In [ ]:
dengue_ok=dengue_ok[~dengue_ok.provincia.str.contains(r'[^A-Z\sÑ]',regex=True)]

What levels of this?

In [ ]:
dengue_ok.enfermedad.value_counts()

What do we have here?

In [ ]:
dengue_ok.tipo_edad.value_counts()

Are YEARS ok?

In [ ]:
dengue_ok.ano.value_counts().plot(kind='bar')

What about AGES ?

In [ ]:
dengue_ok.edad.describe().apply(lambda x: format(x, 'f'))

In [ ]:
# verifying:
dengue_ok[dengue_ok.edad>110]

Let's make the last change and save:

In [ ]:
toDelete=['diagnostic',	'localcod']
dengue_ok.drop(columns=toDelete,inplace=True)
dengue_ok=dengue_ok[dengue_ok.edad<110]

import os

folder = "data"

# Check whether the specified path exists or not
if not os.path.exists(folder):
   # Create a new directory because it does not exist
   os.makedirs(folder)
   dengue_ok.to_csv(os.path.join('data','dengue_ok.csv'),index=False)
   dengue_ok.to_parquet(os.path.join('data','dengue_ok.parquet'),index=False)

else:
    dengue_ok.to_csv(os.path.join('data','dengue_ok.csv'),index=False)
    dengue_ok.to_parquet(os.path.join('data','dengue_ok.parquet'),index=False)